<a href="https://colab.research.google.com/github/S3330/DL2021/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the dataset
# Google Drive
!gdown --id '17Bt6c7lXt3vMRc7ZR56QWt3fXKrVtngz' --output bird.zip
!unzip -q bird.zip

Downloading...
From: https://drive.google.com/uc?id=17Bt6c7lXt3vMRc7ZR56QWt3fXKrVtngz
To: /content/bird.zip
100% 681M/681M [00:04<00:00, 146MB/s]


In [2]:
import numpy as np
import pandas as pd
import shutil, os
import matplotlib.pyplot as plt 
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import PIL
from PIL import Image
import cv2

from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

from tqdm.auto import tqdm

In [3]:
# create subfolders for training images
labels = pd.read_csv(
    'training_labels.txt', sep=" ",header=None)
labels.columns=['name', 'class']
labels = labels.sort_values('class')
class_names = list(labels['class'].unique())

for i in class_names:
  os.makedirs(os.path.join('train_temp', i))

for c in class_names: # category Name
  for i in list(labels[labels['class']==c]['name']): # image Id
    get_image = os.path.join('training_images/', i) # path to images
    move_image_to_cat = shutil.move(get_image, 'train_temp/'+c)

In [4]:
# spilt training and validation set
!pip install split-folders
import splitfolders
splitfolders.ratio('train_temp', output='train_and_val', seed=1337, ratio=(.8, 0.2)) 

Copying files: 3000 files [00:01, 2221.56 files/s]


In [5]:
# create a subfolder named 00 for testing images
os.makedirs(os.path.join('testing_images', '00'))
file_names = os.listdir('testing_images')

for file_name in file_names:
  if file_name != '00':
    shutil.move(os.path.join('testing_images', file_name), 'testing_images/00')

In [6]:
# delete redundant folders and rename folders
shutil.rmtree('train_temp')
shutil.rmtree('training_images')
os.rename('testing_images', 'test')

In [7]:
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [8]:
batch_size = 200

# Construct datasets.
train_set = DatasetFolder('train_and_val/train', loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder('train_and_val/val', loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
test_set = DatasetFolder('test', loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
# change output dimension to 200
import torchvision.models as models
resnet18 = models.resnet18(pretrained = True)
in_ftr  = resnet18.fc.in_features
out_ftr = 200
resnet18.fc = nn.Linear(in_ftr,out_ftr,bias=True)
# inception = models.inception_v3(pretrained = True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [11]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = resnet18.to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# The number of training epochs.
n_epochs = 10

for epoch in range(n_epochs):
    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    # if (i+1) == 1000:
    #   save model

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/12 [00:00<?, ?it/s]

[ Train | 001/010 ] loss = 3.30765, acc = 0.49833


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 001/010 ] loss = 4.26466, acc = 0.15333


...

[ Valid | 009/010 ] loss = 3.42128, acc = 0.30833


  0%|          | 0/12 [00:00<?, ?it/s]

[ Train | 010/010 ] loss = 0.02672, acc = 0.99750


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 010/010 ] loss = 3.41253, acc = 0.29833


In [18]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
with open('testing_img_order.txt') as f:
  test_images = [x.strip() for x in f.readlines()]  # all the testing images
test_images.sort()
test_images

In [ ]:
num = list(range(0,200))
num_to_class_dict = dict(zip(num, class_names))
num_to_class_dict

In [ ]:
predicted_class = []
for i in range(len(predictions)):
  predicted_class.append(num_to_class_dict[predictions[i]])
predicted_class

In [ ]:
search_dict = dict(zip(test_images, predicted_class))
search_dict

In [31]:
submission = []
with open('testing_img_order.txt') as f:
  correct_order = [x.strip() for x in f.readlines()]
for i in tqdm(correct_order):
  predicted_class = search_dict[i]  # the predicted category
  submission.append([i, predicted_class])
np.savetxt('answer.txt', submission, fmt='%s')

  0%|          | 0/3033 [00:00<?, ?it/s]